In [360]:
from gspread_dataframe import set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials
import gspread
import pandas as pd
import numpy as np
import json
import sqlite3
import datetime
from dateutil.relativedelta import relativedelta
import datetime
import re

SPREADSHEET_IDS = {
    "EXA FIRST": "10-B_vV1pvUzXmvGAiHhODGJgCloOsAmqSO9HvXpk_T8",
    "コンサートホールエフ成増": "1EDY2RfjDQNsapVrl2X-UrqPKoXrkQmYJnk3uPqccBxY",
    "第一プラザ坂戸1000": "170MVr-BB3LG-g5ItkDT-8TE6R68RW9zJhRfpvQiy-PE",
    "第一プラザみずほ台店": "1_1722pigi_Z1D6eH0tsPfMneGoS9O09fyqD6F-h1mQA",
    "パールショップともえ川越店": "1i70joJ27Hs7inS-D89z9YMSJO1aRvaBeeWn0n9xpktY",
    "パラッツォ川越店": "179nJF0NvLng7xPKsd_NX2pJBXsDNsO8SJhOvUAvFk2I",
    "第一プラザ狭山店": "1IVb2Woq3n_PDZP87LdW9NpFP3Z6LeyQtONCkx_fWIq4",
    "オータ志木駅前店": "",
}
# スプレッドシート認証設定
scope = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/drive",
]
jsonf = r"C:\python\dataOnline\anaslo_02\json\spreeadsheet-347321-ff675ab5ccbd.json"
creds = ServiceAccountCredentials.from_json_keyfile_name(jsonf, scope)
client = gspread.authorize(creds)

In [361]:
def create_df_from_database(hall_name, start_date, end_date, model_name=None):
    # Table name 取得
    DB_PATH = r"C:\python\dataOnline\anaslo_02\db\anaslo_02.db"
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;")
    tables = cursor.fetchall()
    # print(tables)

    cursor.execute(
        "SELECT hall_id, name FROM halls WHERE name LIKE ?", ("%" + hall_name + "%",)
    )
    results = cursor.fetchall()

    # 結果表示
    if results:
        hall_id, hall_name = results[0]
        print(f"🔍 '{hall_name}' を含むホール名が見つかりました。")
    else:
        print(f"❌ '{hall_name}' を含むホール名は見つかりませんでした。")

    query = """
        -- 出玉データにホール名と機種名を結合して取得
        SELECT
            r.*, 
            h.name AS hall_name,
            m.name AS model_name
        FROM results r
        JOIN halls h ON r.hall_id = h.hall_id
        JOIN models m ON r.model_id = m.model_id
        WHERE h.name = ?
        AND r.date BETWEEN ? AND ?
        """

    params = [hall_name, start_date, end_date]
    if model_name:
        query += " AND m.name LIKE ?"
        params.append(f"%{model_name}%")  # 部分一致にする

    query += " ORDER BY r.date DESC, r.unit_no ASC"

    df = pd.read_sql_query(query, conn, params=params)
    conn.close()
    print(f"データサイズ: {df.shape[0]} x {df.shape[1]}")
    print(f"📅 検索期間: {start_date} ～ {end_date}", f"📅 抽出期間: {df.date.min()} ～ {df.date.max()}")   

    return df

In [399]:
# ブドウシミュレーター
def grape_calc_myfive(game, bb, rb, medals, cherry=True):
    bb_medals = 239.25
    rb_medals = 95.25
    replay_rate = 0.411
    if cherry:
        cherry_rate_high = 0.04228
    else:
        cherry_rate_high = 0.05847
    denominator_inner = (
        -medals
        - (
            game * 3
            - (
                bb * bb_medals
                + rb * rb_medals
                + game * replay_rate
                + game * cherry_rate_high
            )
        )
    ) / 8
    grape_rate = (game / denominator_inner) - ((game / denominator_inner) * 2)
    return grape_rate


GRAPE_CONSTANTS = {
    "マイジャグラーV": {
        "bb": 239.25,
        "rb": 95.25,
        "replay": 0.411,
        "cherryOn": 0.04228,
        "cherryOff": 0.04228,
    },
    "アイムジャグラーEX-TP": {
        "bb": 251.25,
        "rb": 95.25,
        "replay": 0.411,
        "cherryOn": 0.040475,
        "cherryOff": 0.040475,
    },
    "ゴーゴージャグラー3": {
        "bb": 239.00,
        "rb": 95.00,
        "replay": 0.411,
        "cherryOn": 0.0372,
        "cherryOff": 0.0372,
    },
    "ファンキージャグラー2": {
        "bb": 239.25,
        "rb": 95.25,
        "replay": 0.411,
        "cherryOn": 0.04324,
        "cherryOff": 0.04324,
    },
}


def calc_grape_rate(row, constants, cherry=True):
    model = row["model_name"]
    if model not in constants:
        return None
    try:
        game = row["game"]
        bb = row["BB"]
        rb = row["RB"]
        medals = row["medals"]
        # 定数取得
        c = constants[model]
        cherry_rate = c["cherryOn"] if cherry else c["cherryOff"]
        # 分母計算式
        denominator = (
            -medals
            - (
                game * 3
                - (
                    bb * c["bb"]
                    + rb * c["rb"]
                    + game * c["replay"]
                    + game * cherry_rate
                )
            )
        ) / 8
        if denominator == 0:
            return None  # ゼロ除算防止
        grape = (game / denominator) - ((game / denominator) * 2)
        return round(grape, 2)

    except Exception as e:
        print(f"⚠️ Grape計算失敗: {model} → {e}")
        return None


def assign_area(unit_no, json_file_path):
    with open(json_file_path, "r", encoding="utf-8") as f:
        area_map = json.load(f)
    for rule in area_map:
        if rule["start"] <= unit_no <= rule["end"]:
            return rule["area"]
    return "その他"


def df_preprocessing(df, hall_name):
    json_path = f"C:/python/dataOnline/anaslo_02/json/{hall_name}_area_map.json"
    print(f"データ前処理を行います")
    df_pre = df.copy()
    df_pre["date"] = pd.to_datetime(df_pre["date"])
    df_pre.drop(columns=["result_id", "hall_id", "model_id"], inplace=True)
    df_pre_columns = ["hall_name", "date", "model_name", "unit_no", "game", "BB", "RB", "medals"]
    df_pre = df_pre[df_pre_columns]
    df_pre["BB_rate"] = (df_pre["game"] / df_pre["BB"]).round(1)
    df_pre["RB_rate"] = (df_pre["game"] / df_pre["RB"]).round(1)
    df["Grape_rate"] = grape_calc_myfive(
        df["game"], df["BB"], df["RB"], df["medals"], cherry=True
    ).round(2)
    df_pre["Total_rate"] = (df_pre["game"] / (df_pre["BB"] + df_pre["RB"])).round(1)
    df_pre["month"] = df_pre["date"].dt.strftime("%Y-%m")
    df_pre["day"] = df_pre["date"].dt.day
    df_pre["weekday"] = df_pre["date"].dt.weekday
    df_pre["year"] = df_pre["date"].dt.year
    df_pre["unit_last"] = df_pre["unit_no"].astype(str).str[-1]

    df_pre["area"] = df_pre["unit_no"].apply(lambda x: assign_area(x, json_path))

    df_pre = df_pre.replace([np.inf, -np.inf], np.nan)
    df_pre = df_pre.fillna(0)

    model_list = list(df["model_name"].unique())
    for i, model in enumerate(model_list):
        print(f"{i+1}: {model}", end=", ")

    return df_pre, model_list

## ピボットテーブル作成


In [ ]:
def create_pivot_table(
    df, index, columns, pivots=["game", "medals", "BB", "RB"], reverse=False, margins=True, day_target=None
    ):
    df_filtered = df.copy()
    if day_target is not None:
        df_filtered = df_filtered[df_filtered["day"] == day_target]
    
    pivot_results = {}
    for col in pivots:
        table = df_filtered.pivot_table(
            index=index,
            columns=columns,
            values=col,
            aggfunc="sum",
            margins=margins,
            margins_name="total",
        )
        pivot_results[col] = table
        if reverse:
            pivot_results[col] = table.iloc[:, ::-1]

    game = pivot_results["game"]
    medals = pivot_results["medals"]
    rb = pivot_results["RB"]
    bb = pivot_results["BB"]
    rb_rate = (game / rb).round(1)
    total_rate = (game / (bb + rb)).round(1)
    medal_rate = ((medals + game * 3) / (game * 3)).round(3)

    labeled_tables = [
        ("GAME", game),
        ("MEDALS", medals),
        ("RB_RATE", rb_rate),
        ("TOTAL_RATE", total_rate),
        ("MEDAL_RATE", medal_rate),
        ("BB", bb),
        ("RB", rb),
    ]

    # ラベルを MultiIndex に付ける
    for label, df_table in labeled_tables:
        df_table.columns = pd.MultiIndex.from_product([[label], df_table.columns])

    # 列を交互に整列して統合
    interleaved_cols = [
        col
        for pair in zip(
            game.columns,
            medals.columns,
            # bb.columns,
            # rb.columns,
            medal_rate.columns,
            rb_rate.columns,
            total_rate.columns,
        )
        for col in pair
    ]
    merged = pd.concat([game, medals, medal_rate, rb_rate, total_rate], axis=1)[
        interleaved_cols
    ]
    merged.replace([np.inf, -np.inf, np.nan], None, inplace=True)
    details = {
        "game": game, "medals": medals, "medal_rate": medal_rate,
        "bb": bb, "rb": rb, "rb_rate": rb_rate, "total_rate": total_rate,
    }

    return merged, details


def add_spreadsheet(spreadsheet, sheet_name, df, sheet_clear=False):
    today = datetime.datetime.today()
    rows, cols = df.shape
    try:
        sheet = spreadsheet.worksheet(sheet_name)
        print(f"✅ シート「{sheet_name}」が既に存在します。")
    except gspread.exceptions.WorksheetNotFound:
        sheet = spreadsheet.add_worksheet(title=sheet_name, rows=str(rows+3), cols=str(cols+3))
        print(f"🆕 シート「{sheet_name}」を新規作成しました。")
    if sheet_clear:
        sheet.clear()
    last_row = len(sheet.get_all_values())
    set_with_dataframe(sheet, df, row=last_row, include_index=True)
    sheet.update_cell(1, 1, today.strftime("%Y-%m-%d UPDATED"))
    print(f"✅ シート '{sheet_name}' に DataFrame を書き込みました！")

## 出力データ

- 期間指定
  - 機種別 x 日付
  - 島別 x 日付
  - 台番号 x 日付
  - 島別 x 月
  - 過去 n 日の差枚比較

In [419]:
hall_name = "EXA FIRST"
# hall_name = "コンサートホールエフ成増"
# hall_name = "第一プラザみずほ台店"
# hall_name = "オータ志木駅前店"
# hall_name = "第一プラザ坂戸1000"

model_name = "ジャグラー"

today = datetime.date.today()
start_date = today - relativedelta(months=6, days=today.day - 1)
df_db = create_df_from_database(hall_name, start_date, today, model_name=model_name)
df, model_list = df_preprocessing(df_db, hall_name)
df.head()

🔍 'EXA FIRST' を含むホール名が見つかりました。
データサイズ: 30400 x 11
📅 検索期間: 2024-11-01 ～ 2025-05-22 📅 抽出期間: 2024-11-01 ～ 2025-05-21
データ前処理を行います
1: ゴーゴージャグラー3, 2: マイジャグラーV, 3: ファンキージャグラー2, 4: アイムジャグラーEX-TP, 5: ミスタージャグラー, 6: ウルトラミラクルジャグラー, 7: ハッピージャグラーVIII, 8: ジャグラーガールズ, 

,hall_name,date,model_name,unit_no,game,BB,RB,medals,BB_rate,RB_rate,Total_rate,month,day,weekday,year,unit_last,area
0,EXA FIRST,2025-05-21,ゴーゴージャグラー3,1001,5013,16,17,-768,313.3,294.9,151.9,2025-05,21,2,2025,1,a: ゴーゴージャグラー3
1,EXA FIRST,2025-05-21,ゴーゴージャグラー3,1002,2270,8,7,-209,283.8,324.3,151.3,2025-05,21,2,2025,2,a: ゴーゴージャグラー3
2,EXA FIRST,2025-05-21,ゴーゴージャグラー3,1003,8640,30,36,-88,288.0,240.0,130.9,2025-05,21,2,2025,3,a: ゴーゴージャグラー3
3,EXA FIRST,2025-05-21,ゴーゴージャグラー3,1004,1857,9,5,232,206.3,371.4,132.6,2025-05,21,2,2025,4,a: ゴーゴージャグラー3
4,EXA FIRST,2025-05-21,ゴーゴージャグラー3,1005,4248,15,13,-332,283.2,326.8,151.7,2025-05,21,2,2025,5,a: ゴーゴージャグラー3


In [393]:
spreadsheet = client.open_by_key(SPREADSHEET_IDS[hall_name])
print(f"スプレッドシート: {spreadsheet.title} を開きました。")

スプレッドシート: 第一プラザ坂戸1000 を開きました。


## MODEL_RATE


In [ ]:

sheet_name = "MODEL_RATE"
csv_path = f"C:/python/dataOnline/anaslo_02/out/{hall_name}_{sheet_name}.csv"
index = ["model_name"]
columns = ["day"]
merged, details = create_pivot_table(df, index, columns, reverse=False, margins=True)
model_rate = details["medal_rate"].copy()
model_rate.to_csv(csv_path)
# add_spreadsheet(spreadsheet, sheet_name, model_rate, sheet_clear=True)
model_rate.head()

✅ シート「MODEL_RATE」が既に存在します。
✅ シート 'MODEL_RATE' に DataFrame を書き込みました！


MEDAL_RATE                                                   \
day                    1      2      3      4      5      6      7      8   
model_name                                                                  
アイムジャグラーEX-TP      0.993  0.992  0.979  0.978  0.980  0.981  1.006  0.983   
ウルトラミラクルジャグラー      0.981  0.997  0.944  0.963  0.990  1.008  0.977  0.998   
ゴーゴージャグラー3         1.019  0.985  0.985  0.971  1.013  0.985  0.957  0.995   
ジャグラーガールズ          0.968  0.942  1.010  0.959  0.995  0.957  0.980  0.970   
ハッピージャグラーVIII      0.962  0.951  0.950  0.953  0.957  0.997  0.970  0.982   

                             ...                                            \
day                9     10  ...     23     24     25     26     27     28   
model_name                   ...                                             
アイムジャグラーEX-TP  0.981  0.986  ...  0.965  0.993  0.979  0.971  0.962  0.975   
ウルトラミラクルジャグラー  0.957  0.938  ...  0.987  0.990  0.943  1.026  0.982  1.005   
ゴーゴージャグラー3     0.989  0.985  ...  0.969  0.982  0.988  1.000  0.972  0.993   
ジャグラーガールズ      0.971  0.966  ...  0.964  0.964  0.978  0.977  0.981  0.994   
ハッピージャグラーVIII  0.988  0.982  ...  0.987  0.980  1.007  0.968  0.996  1.027   

                                           
day               29     30     31  total  
model_name                                 
アイムジャグラーEX-TP  0.966  0.980  0.991  0.986  
ウルトラミラクルジャグラー  1.026  1.000  0.978  0.987  
ゴーゴージャグラー3     0.996  1.003  0.972  0.992  
ジャグラーガールズ      1.003  0.998  1.003  0.978  
ハッピージャグラーVIII  1.001  0.952  1.026  0.979  

[5 rows x 32 columns]

## ISLAND_RATE


In [ ]:
sheet_name = f"ISLAND_RATE"
csv_path = f"C:/python/dataOnline/anaslo_02/out/{hall_name}_{sheet_name}.csv"
index = ["area"]
columns = ["day"]
merged, details = create_pivot_table(df, index, columns, reverse=False, margins=True)
island_rate = details["medal_rate"].copy()
island_rate.to_csv(csv_path)
# add_spreadsheet(spreadsheet, sheet_name, island_rate, sheet_clear=True)
island_rate.head()

✅ シート「ISLAND_RATE」が既に存在します。
✅ シート 'ISLAND_RATE' に DataFrame を書き込みました！


MEDAL_RATE                                                   \
day                       1      2      3      4      5      6      7      8   
area                                                                           
a: アイムジャグラーEX-TP      0.993  1.021  1.011  0.919  0.881  1.060  0.814  0.992   
b: アイムジャグラーEX-TP      0.986  0.980  0.979  0.957  1.003  0.981  0.991  1.005   
c: アイムジャグラーEX-TP      0.998  0.985  0.945  0.993  0.979  0.977  1.006  0.966   
d: マイジャグラーV           0.992  0.983  0.988  0.981  0.979  0.986  0.972  0.985   
e: ファンキージャグラー2        0.985  0.991  0.975  0.963  0.972  0.998  0.965  0.995   
f: ウルトラミラクルジャグラー      0.981  0.997  0.944  0.963  0.990  1.008  0.977  0.998   
g: ジャグラーガールズ          0.963  0.971  1.000  0.974  0.955  0.933  0.991  0.966   
h: ハッピージャグラーVIII      0.959  0.950  0.961  0.960  0.961  0.997  0.972  0.987   
i: ゴーゴージャグラー3         1.013  0.987  0.989  0.969  1.006  0.985  0.986  0.993   
その他                     NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
total                 0.996  0.984  0.981  0.973  0.988  0.988  0.979  0.988   

                                ...                                            \
day                   9     10  ...     23     24     25     26     27     28   
area                            ...                                             
a: アイムジャグラーEX-TP  0.936  0.849  ...  0.997  0.945  0.909  0.964  0.893  0.981   
b: アイムジャグラーEX-TP  0.974  0.965  ...  0.980  0.971  0.943  0.956  0.973  0.977   
c: アイムジャグラーEX-TP  1.000  0.990  ...  0.967  0.995  1.006  0.996  0.979  0.984   
d: マイジャグラーV       1.000  0.986  ...  0.983  0.993  0.973  0.978  0.994  0.980   
e: ファンキージャグラー2    0.939  0.964  ...  0.928  0.985  0.966  1.026  0.976  0.978   
f: ウルトラミラクルジャグラー  0.957  0.938  ...  0.987  0.990  0.943  1.026  0.982  1.005   
g: ジャグラーガールズ      0.970  0.989  ...  0.937  1.011  0.969  0.962  0.933  1.008   
h: ハッピージャグラーVIII  0.990  0.980  ...  1.000  0.981  1.012  0.973  0.991  1.021   
i: ゴーゴージャグラー3     0.981  0.986  ...  0.965  0.983  0.981  0.987  0.969  0.987   
その他                 NaN    NaN  ...    NaN    NaN    NaN    NaN    NaN    NaN   
total             0.985  0.980  ...  0.972  0.988  0.979  0.989  0.980  0.989   

                                              
day                  29     30     31  total  
area                                          
a: アイムジャグラーEX-TP  0.920  0.971  1.054  0.983  
b: アイムジャグラーEX-TP  0.949  0.969  0.955  0.976  
c: アイムジャグラーEX-TP  0.960  0.982  1.007  0.990  
d: マイジャグラーV       0.976  0.979  0.988  0.986  
e: ファンキージャグラー2    0.977  0.981  0.985  0.979  
f: ウルトラミラクルジャグラー  1.026  1.000  0.978  0.987  
g: ジャグラーガールズ      1.025  0.955  0.985  0.979  
h: ハッピージャグラーVIII  0.994  0.960  1.026  0.980  
i: ゴーゴージャグラー3     0.992  0.999  0.975  0.990  
その他                 NaN    NaN    NaN  0.983  
total             0.984  0.985  0.988  0.986  

[11 rows x 32 columns]

## UNIT_RATE


In [416]:
sheet_name = f"UNIT_RATE"
csv_path = f"C:/python/dataOnline/anaslo_02/out/{hall_name}_{sheet_name}.csv"
index = ["area", "unit_no"]
columns = ["day"]
merged, details = create_pivot_table(df, index, columns, reverse=False, margins=True)
target_rate = 1.05
unit_rate = details["medal_rate"].copy()
unit_rate[("MEDAL_RATE", f"{target_rate}+")] = (
    unit_rate.iloc[:, :-1] >= target_rate
).sum(axis=1)
countif = (unit_rate.iloc[:-1, :] >= target_rate).sum(axis=0)
unit_rate = pd.concat(
    [unit_rate, pd.DataFrame([countif], index=[(f"count_{target_rate}+", "")])], axis=0
)
unit_rate = unit_rate[~unit_rate.iloc[:, 1].isna()]
unit_rate.to_csv(csv_path)
add_spreadsheet(spreadsheet, sheet_name, unit_rate, sheet_clear=True)
unit_rate.head()

✅ シート「UNIT_RATE」が既に存在します。
✅ シート 'UNIT_RATE' に DataFrame を書き込みました！


MEDAL_RATE                                                          \
day               1      2      3      4      5      6      7      8      9   
その他 1001      0.973  1.024  0.995  0.951  0.976  1.018  1.118  0.889  1.016   
    1002      1.051  0.999  1.087  0.950  1.007  1.047  0.814  0.895  1.063   
    1003      1.101  0.909  0.977  1.036  0.975  0.978  0.972  0.630  0.888   
    1004      0.996  0.992  0.887  0.951  0.976  1.001  0.943  0.825  0.996   
    1005      1.015  1.090  1.082  0.937  1.002  0.948  0.885  0.976  1.051   

                 ...                                                          \
day          10  ...     23     24     25     26     27     28     29     30   
その他 1001  1.016  ...  0.794  0.900  1.043  1.088  0.736  0.987  1.048  0.955   
    1002  1.003  ...  1.092  1.088  0.702  1.054  0.952  0.656  0.902  0.872   
    1003  0.937  ...  0.924  1.061  1.069  0.902  1.009  1.049  0.993  0.793   
    1004  1.010  ...  1.029  1.071  0.963  1.056  0.981  0.540  0.931  1.123   
    1005  1.111  ...  0.917  0.984  0.888  1.077  1.047  1.129  0.839  0.809   

                       
day       total 1.05+  
その他 1001  0.990     3  
    1002  1.002     9  
    1003  1.009     5  
    1004  0.991     5  
    1005  1.006     7  

[5 rows x 32 columns]

# DAYn

- SHEET_NAME : DAY1，DAY2，… DAY31


In [413]:
index = ["area", "unit_no"]
columns = ["date"]
day_list = [3, 6, 7, 8, 9, 11, 13, 19, 21, 26, 19, 31]
# for day_target in range(today.day - 1, today.day + 1):
for day_target in day_list:
    sheet_name = f"DAY{day_target}"
    csv_path = f"C:/python/dataOnline/anaslo_02/out/{hall_name}_{sheet_name}.csv"
    merged, details = create_pivot_table(df, index, columns, reverse=True, margins=True, day_target=day_target)
    merged.to_csv(csv_path)
    add_spreadsheet(spreadsheet, f"DAY{day_target}", merged, sheet_clear=True,)
merged.head()

🆕 シート「DAY3」を新規作成しました。
✅ シート 'DAY3' に DataFrame を書き込みました！
🆕 シート「DAY6」を新規作成しました。
✅ シート 'DAY6' に DataFrame を書き込みました！
🆕 シート「DAY7」を新規作成しました。
✅ シート 'DAY7' に DataFrame を書き込みました！
🆕 シート「DAY8」を新規作成しました。
✅ シート 'DAY8' に DataFrame を書き込みました！
🆕 シート「DAY9」を新規作成しました。
✅ シート 'DAY9' に DataFrame を書き込みました！
🆕 シート「DAY11」を新規作成しました。
✅ シート 'DAY11' に DataFrame を書き込みました！
🆕 シート「DAY13」を新規作成しました。
✅ シート 'DAY13' に DataFrame を書き込みました！
🆕 シート「DAY19」を新規作成しました。
✅ シート 'DAY19' に DataFrame を書き込みました！
🆕 シート「DAY21」を新規作成しました。
✅ シート 'DAY21' に DataFrame を書き込みました！
🆕 シート「DAY26」を新規作成しました。
✅ シート 'DAY26' に DataFrame を書き込みました！
✅ シート「DAY19」が既に存在します。
✅ シート 'DAY19' に DataFrame を書き込みました！
🆕 シート「DAY31」を新規作成しました。
✅ シート 'DAY31' に DataFrame を書き込みました！


GAME MEDALS MEDAL_RATE RB_RATE TOTAL_RATE  \
date                     total  total      total   total      total   
area             unit_no                                              
a: アイムジャグラーEX-TP 663      3532    849       1.08   321.1      126.1   
                 664       374   -168       0.85   374.0      187.0   
                 665        33    -42      0.576    None       None   
                 666         0      0       None    None       None   
b: アイムジャグラーEX-TP 856      5077  -1180      0.923   362.6      175.1   

                                        GAME              MEDALS  \
date                     2025-03-31 00:00:00 2025-03-31 00:00:00   
area             unit_no                                           
a: アイムジャグラーEX-TP 663                     0.0                 0.0   
                 664                    29.0               -46.0   
                 665                    33.0               -42.0   
                 666                     0.0                 0.0   
b: アイムジャグラーEX-TP 856                  2569.0              -680.0   

                                  MEDAL_RATE             RB_RATE  \
date                     2025-03-31 00:00:00 2025-03-31 00:00:00   
area             unit_no                                           
a: アイムジャグラーEX-TP 663                    None                None   
                 664                   0.471                None   
                 665                   0.576                None   
                 666                    None                None   
b: アイムジャグラーEX-TP 856                   0.912               428.2   

                                  TOTAL_RATE                GAME  \
date                     2025-03-31 00:00:00 2025-01-31 00:00:00   
area             unit_no                                           
a: アイムジャグラーEX-TP 663                    None              3532.0   
                 664                    None               345.0   
                 665                    None                None   
                 666                    None                None   
b: アイムジャグラーEX-TP 856                   183.5              1523.0   

                                      MEDALS          MEDAL_RATE  \
date                     2025-01-31 00:00:00 2025-01-31 00:00:00   
area             unit_no                                           
a: アイムジャグラーEX-TP 663                   849.0                1.08   
                 664                  -122.0               0.882   
                 665                    None                None   
                 666                    None                None   
b: アイムジャグラーEX-TP 856                  -203.0               0.956   

                                     RB_RATE          TOTAL_RATE  \
date                     2025-01-31 00:00:00 2025-01-31 00:00:00   
area             unit_no                                           
a: アイムジャグラーEX-TP 663                   321.1               126.1   
                 664                   345.0               172.5   
                 665                    None                None   
                 666                    None                None   
b: アイムジャグラーEX-TP 856                   507.7               190.4   

                                        GAME              MEDALS  \
date                     2024-12-31 00:00:00 2024-12-31 00:00:00   
area             unit_no                                           
a: アイムジャグラーEX-TP 663                    None                None   
                 664                    None                None   
                 665                    None                None   
                 666                    None                None   
b: アイムジャグラーEX-TP 856                   985.0              -297.0   

                                  MEDAL_RATE             RB_RATE  \
date                     2024-12-31 00:00:00 2024-12-31 00:00:00   
area             unit_no                           

## MONTH_RATE

In [ ]:
start_date = today - relativedelta(months=100, days=today.day-1)
df_db = create_df_from_database("EXA FIRST", start_date, today, model_name=model_name)
df, model_list = df_preprocessing(df_db, hall_name)
sheet_name = "MONTH_RATE"
csv_path = f"C:/python/dataOnline/anaslo_02/out/{hall_name}_{sheet_name}.csv"
index = ["model_name"]
columns = ["month"]
merged, details = create_pivot_table(df, index, columns, reverse=True, margins=True)
month_rate = details["medal_rate"].copy()
add_spreadsheet(spreadsheet, sheet_name, month_rate, sheet_clear=True)
month_rate.to_csv(csv_path)
month_rate

🔍 'EXA FIRST' を含むホール名が見つかりました。
データサイズ: 62928 x 11
📅 検索期間: 2017-01-01 ～ 2025-05-22 📅 抽出期間: 2024-04-01 ～ 2025-05-21
データ前処理を行います
1: ゴーゴージャグラー3, 2: マイジャグラーV, 3: ファンキージャグラー2, 4: アイムジャグラーEX-TP, 5: ミスタージャグラー, 6: ウルトラミラクルジャグラー, 7: ハッピージャグラーVIII, 8: ジャグラーガールズ, 

MEDAL_RATE                                                  \
month              total 2025-05 2025-04 2025-03 2025-02 2025-01 2024-12   
model_name                                                                 
アイムジャグラーEX-TP      1.002   0.996   0.998   1.001   0.993   0.997   1.000   
ウルトラミラクルジャグラー      0.998   0.987   0.993   0.997   1.002   1.000   1.001   
ゴーゴージャグラー3         1.002   0.999   0.993   0.992   1.001   1.000   0.996   
ジャグラーガールズ          1.003   1.000   0.993   1.001   0.990   0.996   0.994   
ハッピージャグラーVIII      0.995   0.993   0.995     NaN     NaN     NaN     NaN   
ファンキージャグラー2        0.996   0.992   0.992   0.993   0.996   0.990   0.995   
マイジャグラーV           1.010   1.002   1.000   1.002   1.003   1.006   1.005   
ミスタージャグラー          1.001   0.999   1.002   0.991   0.996   0.990   1.006   
total              1.006   0.999   0.998   0.999   1.001   1.002   1.002   

                                                                               
month         2024-11 2024-10 2024-09 2024-08 2024-07 2024-06 2024-05 2024-04  
model_name                                                                     
アイムジャグラーEX-TP   1.001   0.999   1.001   1.000   1.007   1.005   1.004   1.008  
ウルトラミラクルジャグラー     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN  
ゴーゴージャグラー3      1.005   0.995   0.998   1.003   1.003   1.006   1.010   1.012  
ジャグラーガールズ       0.995   0.999   1.003   0.997   1.005   1.005   1.010   1.016  
ハッピージャグラーVIII   0.957   1.004     NaN     NaN     NaN     NaN     NaN     NaN  
ファンキージャグラー2     0.996   1.000   0.989   1.001   0.986   0.990   1.000   1.007  
マイジャグラーV        1.002   1.008   1.009   1.012   1.011   1.017   1.021   1.025  
ミスタージャグラー       0.997   1.002   1.008   1.007   1.009     NaN     NaN     NaN  
total           1.001   1.004   1.005   1.007   1.008   1.011   1.015   1.018

## HISTORY_ALL

In [417]:
start_date = today - relativedelta(months=1, days=today.day-1)
df_db = create_df_from_database("EXA FIRST", start_date, today, model_name=model_name)
df, model_list = df_preprocessing(df_db, hall_name)

sheet_name = "HISTORY_ALL"
index = ["area", "unit_no"]
columns = ["date"]
merged, details = create_pivot_table(df, index, columns, reverse=True, margins=True)
add_spreadsheet(spreadsheet, sheet_name, merged, sheet_clear=True)
csv_path = f"C:/python/dataOnline/anaslo_02/out/{hall_name}_{sheet_name}.csv"
merged.to_csv(csv_path)
merged.head()

🔍 'EXA FIRST' を含むホール名が見つかりました。
データサイズ: 7752 x 11
📅 検索期間: 2025-04-01 ～ 2025-05-22 📅 抽出期間: 2025-04-01 ～ 2025-05-21
データ前処理を行います
1: ゴーゴージャグラー3, 2: マイジャグラーV, 3: ファンキージャグラー2, 4: アイムジャグラーEX-TP, 5: ミスタージャグラー, 6: ウルトラミラクルジャグラー, 7: ハッピージャグラーVIII, 8: ジャグラーガールズ, 🆕 シート「HISTORY_ALL」を新規作成しました。
✅ シート 'HISTORY_ALL' に DataFrame を書き込みました！


GAME MEDALS MEDAL_RATE RB_RATE TOTAL_RATE                GAME  \
date           total  total      total   total      total 2025-05-21 00:00:00   
area unit_no                                                                    
その他  1001     249294  -7435       0.99   331.9      146.6                5013   
     1002     171153   1267      1.002   307.8      141.6                2270   
     1003     175873   4881      1.009   322.1      140.7                8640   
     1004     176139  -4681      0.991   356.6      149.0                1857   
     1005     224092   4070      1.006   308.2      140.0                4248   

                          MEDALS          MEDAL_RATE             RB_RATE  \
date         2025-05-21 00:00:00 2025-05-21 00:00:00 2025-05-21 00:00:00   
area unit_no                                                               
その他  1001                   -768               0.949               294.9   
     1002                   -209               0.969               324.3   
     1003                    -88               0.997               240.0   
     1004                    232               1.042               371.4   
     1005                   -332               0.974               326.8   

                      TOTAL_RATE  ...                GAME              MEDALS  \
date         2025-05-21 00:00:00  ... 2025-04-02 00:00:00 2025-04-02 00:00:00   
area unit_no                      ...                                           
その他  1001                  151.9  ...                5376                 838   
     1002                  151.3  ...                3486                 221   
     1003                  130.9  ...                1224                -326   
     1004                  132.6  ...                 862                -426   
     1005                  151.7  ...                6749                2621   

                      MEDAL_RATE             RB_RATE          TOTAL_RATE  \
date         2025-04-02 00:00:00 2025-04-02 00:00:00 2025-04-02 00:00:00   
area unit_no                                                               
その他  1001                  1.052               413.5               137.8   
     1002                  1.021               193.7               116.2   
     1003                  0.911               612.0               204.0   
     1004                  0.835               431.0               215.5   
     1005                  1.129               270.0               114.4   

                            GAME              MEDALS          MEDAL_RATE  \
date         2025-04-01 00:00:00 2025-04-01 00:00:00 2025-04-01 00:00:00   
area unit_no                                                               
その他  1001                   6879                 344               1.017   
     1002                   1441                -450               0.896   
     1003                   7691                3306               1.143   
     1004                   4512                 226               1.017   
     1005                   1852                -162               0.971   

                         RB_RATE          TOTAL_RATE  
date         2025-04-01 00:00:00 2025-04-01 00:00:00  
area unit_no                                          
その他  1001                  286.6               143.3  
     1002                  480.3               180.1  
     1003                  427.3               128.2  
     1004                  410.2               150.4  
     1005                 1852.0               185.2  

[5 rows x 260 columns]

## HISTORY_UNIT_RATE

In [ ]:
start_date = today - relativedelta(months=0, days=today.day-1)
df_db = create_df_from_database("EXA FIRST", start_date, today, model_name=model_name)
df, model_list = df_preprocessing(df_db, hall_name)

sheet_name = "HISTORY_UNIT_RATE"
index = ["area", "unit_no"]
columns = ["date"]
merged, details = create_pivot_table(df, index, columns, reverse=True, margins=True)
csv_path = f"C:/python/dataOnline/anaslo_02/out/{hall_name}_{sheet_name}_merged.csv"
unit_rate_history = details["medal_rate"].copy()
unit_rate_history.to_csv(csv_path)
unit_rate_history.head()

🔍 'EXA FIRST' を含むホール名が見つかりました。
データサイズ: 3192 x 11
📅 検索期間: 2025-05-01 ～ 2025-05-22 📅 抽出期間: 2025-05-01 ～ 2025-05-21
データ前処理を行います
1: ゴーゴージャグラー3, 2: マイジャグラーV, 3: ファンキージャグラー2, 4: アイムジャグラーEX-TP, 5: ミスタージャグラー, 6: ウルトラミラクルジャグラー, 7: ハッピージャグラーVIII, 8: ジャグラーガールズ, ✅ シート「HISTORY_UNIT_RATE」が既に存在します。
✅ シート 'HISTORY_UNIT_RATE' に DataFrame を書き込みました！


MEDAL_RATE                                          \
date                       total 2025-05-21 00:00:00 2025-05-20 00:00:00   
area          unit_no                                                      
a: ゴーゴージャグラー3 1001         1.003               0.949               0.902   
              1002         0.997               0.969               0.869   
              1003         1.006               0.997               0.912   
              1004         0.988               1.042               0.982   
              1005         1.000               0.974               0.927   

                                                               \
date                  2025-05-19 00:00:00 2025-05-18 00:00:00   
area          unit_no                                           
a: ゴーゴージャグラー3 1001                  0.988               0.873   
              1002                  0.775               1.013   
              1003                  0.777               1.070   
              1004                  0.923               0.794   
              1005                  1.162               0.941   

                                                               \
date                  2025-05-17 00:00:00 2025-05-16 00:00:00   
area          unit_no                                           
a: ゴーゴージャグラー3 1001                  1.053               0.997   
              1002                  0.859               0.802   
              1003                  0.698               0.695   
              1004                  1.009               1.018   
              1005                  1.057               0.964   

                                                               \
date                  2025-05-15 00:00:00 2025-05-14 00:00:00   
area          unit_no                                           
a: ゴーゴージャグラー3 1001                  0.935               1.101   
              1002                  1.083               1.001   
              1003                  0.792               1.025   
              1004                  1.110               1.082   
              1005                  0.958               0.819   

                                           ...                      \
date                  2025-05-13 00:00:00  ... 2025-05-10 00:00:00   
area          unit_no                      ...                       
a: ゴーゴージャグラー3 1001                  1.153  ...               1.082   
              1002                  0.578  ...               0.975   
              1003                  1.144  ...               0.941   
              1004                  1.059  ...               0.891   
              1005                  0.802  ...               1.109   

                                                               \
date                  2025-05-09 00:00:00 2025-05-08 00:00:00   
area          unit_no                                           
a: ゴーゴージャグラー3 1001                  0.921               0.887   
              1002                  0.964               0.918   
              1003                  0.891               0.550   
              1004                  1.028               0.849   
              1005                  1.107               0.957   

                                                               \
date                  2025-05-07 00:00:00 2025-05-06 00:00:00   
area          unit_no                                           
a: ゴーゴージャグラー3 1001                  1.183               0.973   
              1002                  0.842               1.000   
              1003                  0.966               0.871   
              1004                  0.914               1.034   
              1005                  0.911               0.890   

                                                               \
date                  2025-05-05 00:00:00 2025-05-04 00:00:00   
area          unit_no                                           
a: ゴーゴージャグラー3 1001                  0.973           